In [2]:
from helper import download_dataset
import pandas as pd

In [3]:
train_pd, valid_pd, test_pd = download_dataset()

Downloaded storage object train.json from bucket watclaimcheck_dataset.
Downloaded storage object valid.json from bucket watclaimcheck_dataset.
Downloaded storage object test.json from bucket watclaimcheck_dataset.


In [4]:
train_pd

,metadata,label
0,"{'claimant': 'Faisal Al Qasimi, Carolina Monte...","{'reviewer_name': 'Alt News', 'reviewer_site':..."
1,"{'claimant': 'Joe Biden', 'claim': '“38,000 pr...","{'reviewer_name': 'FactCheck.org', 'reviewer_s..."
2,"{'claimant': 'Tony Evers', 'claim': 'Says Wisc...","{'reviewer_name': 'PolitiFact', 'reviewer_site..."
3,"{'claimant': None, 'claim': 'A 4-year-old boy ...","{'reviewer_name': 'Snopes', 'reviewer_site': '..."
4,"{'claimant': 'Multiple sources', 'claim': 'Whi...","{'reviewer_name': 'AFP Fact Check', 'reviewer_..."
...,...,...
26971,"{'claimant': 'Michael Chippendale', 'claim': '...","{'reviewer_name': 'PolitiFact', 'reviewer_site..."
26972,"{'claimant': None, 'claim': 'Former NBA star s...","{'reviewer_name': 'Snopes', 'reviewer_site': '..."
26973,"{'claimant': 'Adam Kinzinger', 'claim': '""Sinc...","{'reviewer_name': 'PolitiFact', 'reviewer_site..."
26974,"{'claimant': 'Joe Lieberman', 'claim': 'Presid...","{'reviewer_name': 'PolitiFact', 'reviewer_site..."
